MongoDB에서 label_math컬렉션을 가져옴

In [1]:
import pandas as pd
from pymongo import MongoClient
from bson import ObjectId

# MongoDB 연결 설정
username = 'root'
password = 'qwe123'
host = '127.0.0.1'
port = 27017

# MongoDB 클라이언트 연결
client = MongoClient(f'mongodb://{username}:{password}@{host}:{port}/')
db = client['project']
collection = db['math_knowledge_data_set']

def mongo_to_dataframe():
    # MongoDB에서 데이터 가져오기
    data = list(collection.find({}))  # 모든 데이터를 리스트로 가져오기

    # 데이터가 빈 경우 처리
    if not data:
        return pd.DataFrame()

    # ObjectId를 문자열로 변환
    for entry in data:
        if '_id' in entry and isinstance(entry['_id'], ObjectId):
            entry['_id'] = str(entry['_id'])

    # 데이터를 담을 리스트 초기화
    records = []

    # 모든 데이터를 순회하여 처리
    for entry in data:
        for key, value in entry.items():
            if key.isdigit():  # '0', '1', ..., '3445' 같은 키 값
                from_concept = value.get('fromConcept', {})
                to_concept = value.get('toConcept', {})

                # fromConcept 및 toConcept의 데이터프레임 변환
                from_df = pd.json_normalize(from_concept, sep='_').add_prefix('from_')
                to_df = pd.json_normalize(to_concept, sep='_').add_prefix('to_')

                # 각각의 데이터프레임을 하나의 데이터프레임으로 병합
                combined_df = pd.concat([from_df, to_df], axis=1)

                # ID 컬럼을 추가하여 데이터 식별
                combined_df['id'] = key
                records.append(combined_df)

    # 모든 DataFrame을 하나로 병합
    final_df = pd.concat(records, ignore_index=True)

    return final_df

# 데이터프레임 가져오기
label_math = mongo_to_dataframe()


In [2]:
label_math

,from_id,from_name,from_semester,from_description,from_chapter_id,from_chapter_name,from_achievement_id,from_achievement_name,to_id,to_name,to_semester,to_description,to_chapter_id,to_chapter_name,to_achievement_id,to_achievement_name,id
0,3249,거듭제곱,고등-수1-전체,임의의 수 $a$와 양의 정수 $n$에 대하여 $a$를 $n$개 거듭하여 곱한 것을...,587,지수함수와 로그함수 > 지수 > 거듭제곱과 거듭제곱근,314,"거듭제곱과 거듭제곱근의 뜻을 알고, 그 성질을 설명할 수 있다.",1442,거듭제곱,중등-중2-1학기,같은 수나 문자를 거듭하여 곱한 것을 간단히 나타낸 것,481,식의 계산 > 단항식의 계산 > 지수법칙,88,지수법칙을 이해한다.,0
1,3,지수가 자연수일 때의 지수법칙,고등-수1-전체,"$a$, $b$가 실수이고 $m$, $n$이 자연수일 때\n\n(1)$a^ma^n=...",587,지수함수와 로그함수 > 지수 > 거듭제곱과 거듭제곱근,314,"거듭제곱과 거듭제곱근의 뜻을 알고, 그 성질을 설명할 수 있다.",4659,지수법칙 (2) - 지수의 곱,중등-중2-1학기,"m,n이 자연수일 때\n$(a^m)^n=a^{mn}$",481,식의 계산 > 단항식의 계산 > 지수법칙,88,지수법칙을 이해한다.,1
2,3,지수가 자연수일 때의 지수법칙,고등-수1-전체,"$a$, $b$가 실수이고 $m$, $n$이 자연수일 때\n\n(1)$a^ma^n=...",587,지수함수와 로그함수 > 지수 > 거듭제곱과 거듭제곱근,314,"거듭제곱과 거듭제곱근의 뜻을 알고, 그 성질을 설명할 수 있다.",4660,지수법칙 (3) - 지수의 차,중등-중2-1학기,"$a\ne0$이고 m, n이 자연수일 때\n$a^m\div a^n=\begin{ca...",481,식의 계산 > 단항식의 계산 > 지수법칙,88,지수법칙을 이해한다.,2
3,3,지수가 자연수일 때의 지수법칙,고등-수1-전체,"$a$, $b$가 실수이고 $m$, $n$이 자연수일 때\n\n(1)$a^ma^n=...",587,지수함수와 로그함수 > 지수 > 거듭제곱과 거듭제곱근,314,"거듭제곱과 거듭제곱근의 뜻을 알고, 그 성질을 설명할 수 있다.",4661,지수법칙 (4) - 지수의 분배,중등-중2-1학기,n이 자연수일 때\n1) $(ab)^n=a^nb^n$\n2) $(\frac{a}{b...,481,식의 계산 > 단항식의 계산 > 지수법칙,88,지수법칙을 이해한다.,3
4,4,거듭제곱근,고등-수1-전체,방정식 $x^n=a$의 근 $x$를 $a$의 $n$제곱근이라 한다.\n이때 실수 $...,587,지수함수와 로그함수 > 지수 > 거듭제곱과 거듭제곱근,314,"거듭제곱과 거듭제곱근의 뜻을 알고, 그 성질을 설명할 수 있다.",3249,거듭제곱,고등-수1-전체,임의의 수 $a$와 양의 정수 $n$에 대하여 $a$를 $n$개 거듭하여 곱한 것을...,587,지수함수와 로그함수 > 지수 > 거듭제곱과 거듭제곱근,314,"거듭제곱과 거듭제곱근의 뜻을 알고, 그 성질을 설명할 수 있다.",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3441,11269,천만 단위까지의 수 알아보기,초등-초4-1학기,"10000이 1365개인 수- 쓰기: 13560000또는 1356만, 읽기: 천삼백...",223,"큰 수 > 십만, 백만, 천만을 알아볼까요",11,10000 이상의 큰 수에 대한 자릿값과 위치적 기수법을 원리를 이해한다.,7268,다섯 자릿수,초등-초4-1학기,"1.다섯 자릿수 알아보기\n10000이 2개, 1000이 3개, 100이 8개, 1...",222,큰 수 > 다섯 자릿수를 알아볼까요,10,10000 이상의 큰 수를 읽고 쓸 수 있다.,3441
3442,11270,천억 단위까지의 수 알아보기,초등-초4-1학기,1억이 7365개인 수\n- 쓰기: 736500000000또는 7365억\n- 읽기...,224,큰 수 > 억과 조를 알아볼까요,11,10000 이상의 큰 수에 대한 자릿값과 위치적 기수법을 원리를 이해한다.,11269,천만 단위까지의 수 알아보기,초등-초4-1학기,"10000이 1365개인 수- 쓰기: 13560000또는 1356만, 읽기: 천삼백...",223,"큰 수 > 십만, 백만, 천만을 알아볼까요",11,10000 이상의 큰 수에 대한 자릿값과 위치적 기수법을 원리를 이해한다.,3442
3443,11271,천조 단위까지의 수 알아보기,초등-초4-1학기,1조가 2761개인 수\n- 쓰기: 2761000000000000 또는 2761조\...,224,큰 수 > 억과 조를 알아볼까요,11,10000 이상의 큰 수에 대한 자릿값과 위치적 기수법을 원리를 이해한다.,11270,천억 단위까지의 수 알아보기,초등-초4-1학기,1억이 7365개인 수\n- 쓰기: 736500000000또는 7365억\n- 읽기...,224,큰 수 > 억과 조를 알아볼까요,11,10000 이상의 큰 수에 대한 자릿값과 위치적 기수법을 원리를 이해한다.,3443
3444,8481,각도의 합과 차,초등-초4-1학기,"각도의 합과 차는 자연수의 덧셈, 뺄셈과 같은 방법으로 계산합니다.",232,각도 > 각도의 합과 차는 얼마일까요,NaN,nan,7310,받아 올림이 세 번 있는 $(세자릿수)+(세자릿수)$,초등-초3-1학기,"1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...",148,덧셈과 뺄셈 > 덧셈을 해 볼까요 (3),53,받아 올림이 있는 (세 자릿수)+(세 자릿수)의 계산 원리를 이해하고 그 계산을 할...,3444


label_math 전처리

In [3]:
# from_g와 to_g 열 생성
label_math['from_g'] = label_math['from_semester'].apply(lambda x: x.split('-')[0])
label_math['to_g'] = label_math['to_semester'].apply(lambda x: x.split('-')[0])
# 필터링 조건 적용: 고등 포함 / 중등만 포함한 경우 제외
label_math_ele = label_math[
    ~(
        ((label_math['from_g'] == '고등') | (label_math['to_g'] == '고등')) |
        ((label_math['from_g'] == '중등') & (label_math['to_g'] == '중등'))
    )
]
# 치환을 위한 딕셔너리 생성
replacement_dict = {
    '중등-중1-1학기': '중등-중7-1학기',
    '중등-중2-2학기': '중등-중8-2학기',
    '중등-중1-2학기': '중등-중7-2학기',
    '중등-중2-1학기': '중등-중8-1학기',
    '중등-중3-1학기': '중등-중9-1학기',
    '중등-중3-2학기': '중등-중9-2학기'
}
# 'from_g'가 '중등'인 행들의 'from_semester' 값을 딕셔너리 대로 치환
label_math_ele.loc[label_math_ele['from_g'] == '중등', 'from_semester'] = label_math_ele['from_semester'].replace(replacement_dict)

mongoDB로 부터 JSON파일을 DataFrame으로 변환

In [4]:
from pymongo import MongoClient
import pandas as pd

# MongoDB 연결 설정 (사용자 이름과 비밀번호 포함)
username = 'root'  # MongoDB 사용자 이름
password = 'qwe123'  # MongoDB 사용자 비밀번호
host = '127.0.0.1'  # MongoDB 호스트 주소 (예: localhost)
port = 27017  # MongoDB 포트 (기본: 27017)

# MongoDB 클라이언트 연결 (인증 정보 포함)
client = MongoClient(f'mongodb://{username}:{password}@{host}:{port}/')
db = client['project']  # 데이터베이스 이름 설정

In [5]:
# 컬렉션 선택
qa_collection = db['1_correct_answer']
irt_collection = db['2_questionIRT']
taker_irt_collection = db['3_candidateIRT']

# 데이터를 데이터프레임으로 변환하는 함수
def fetch_collection_as_dataframe(collection):
    data = list(collection.find({}))
    return pd.DataFrame(data)

# 각 컬렉션에서 데이터 가져오기
qa_df = fetch_collection_as_dataframe(qa_collection)
irt_df = fetch_collection_as_dataframe(irt_collection)
taker_irt_df = fetch_collection_as_dataframe(taker_irt_collection)

# 데이터 프레임 병합
merged_df = pd.merge(qa_df, irt_df, on=['testID', 'assessmentItemID'], how='left')
merged_df = pd.merge(merged_df, taker_irt_df, on=['learnerID', 'testID'], how='left')

In [6]:
# 불필요한 컬럼 삭제
columns_to_drop = ['_id_x', 'Timestamp_x', '_id_y', 'Timestamp_y', '_id', 'Timestamp', 'learnerProfile_y']
merged_df = merged_df.drop(columns=columns_to_drop)

# 'learnerProfile_x' 컬럼의 이름을 'learnerProfile'로 변경
merged_df = merged_df.rename(columns={'learnerProfile_x': 'learnerProfile'})
merged_df['knowledgeTag'] = pd.to_numeric(merged_df['knowledgeTag'], errors='coerce', downcast='integer')


In [7]:
merged_df

,learnerID,learnerProfile,testID,assessmentItemID,answerCode,difficultyLevel,discriminationLevel,guessLevel,knowledgeTag,theta,realScore
0,A010000003,F;S01;1,A010000001,A010001001,1,-3.719715,0.721179,1.008904e-02,5844,0.196765,0.967829
1,A010000003,F;S01;1,A010000001,A010001002,1,-2.917577,1.349719,1.278008e-02,5844,0.196765,0.967829
2,A010000003,F;S01;1,A010000001,A010001003,1,-2.273138,1.192981,5.012783e-02,5844,0.196765,0.967829
3,A010000003,F;S01;1,A010000001,A010001004,1,-2.302867,1.732754,6.223898e-03,5844,0.196765,0.967829
4,A010000003,F;S01;1,A010000001,A010001005,1,-1.611647,1.911265,2.348796e-04,5844,0.196765,0.967829
...,...,...,...,...,...,...,...,...,...,...,...
1677353,A090000552,M;S01;9,A090000074,A090074002,1,0.012002,54.381941,2.930000e-08,4243,0.785694,0.706491
1677354,A090000552,M;S01;9,A090000074,A090074003,1,-0.725795,0.416779,1.159106e-02,4243,0.785694,0.706491
1677355,A090000552,M;S01;9,A090000074,A090074004,1,0.450923,4.251689,0.000000e+00,10196,0.785694,0.706491
1677356,A090000552,M;S01;9,A090000074,A090074005,1,2.126145,0.240034,3.483607e-02,2648,0.785694,0.706491


postgresql에서 education_2022 dataframe으로 변환 + 전처리

In [8]:
import pandas as pd
from sqlalchemy import create_engine

# 연결 정보 설정
db_config = {
    'host': "127.0.0.1",
    'port': "5432",
    'user': "postgres",
    'pw': "qwe123",
    'db': "project",
    'location': "localhost_target",
    'engine': "postgre",
    'table_name': "education_2022"
}

# 데이터베이스 URL 구성
db_url = f"postgresql://{db_config['user']}:{db_config['pw']}@{db_config['host']}:{db_config['port']}/{db_config['db']}"

# SQLAlchemy를 사용하여 PostgreSQL에 연결하는 엔진 생성
engine = create_engine(db_url)

# SQL 쿼리 정의 (테이블의 모든 데이터를 선택)
query = f"SELECT * FROM {db_config['table_name']}"

# 쿼리 실행 및 결과를 DataFrame으로 변환
education_2022 = pd.read_sql(query, engine)

# 'Unnamed: 6' 컬럼을 제거
education_2022 = education_2022.drop(columns=['Unnamed: 6'], errors='ignore')

# '계열화' 컬럼의 NaN 값을 '정보 없음'으로 대체
education_2022['계열화'] = education_2022['계열화'].fillna('정보 없음')


In [9]:
education_2022


,ID,Name,영역명,내용요소,계열화,grade
0,2048,큰 수 > 억과 조를 알아볼까요 / 큰 수 > 억과 조를 알아볼까요,수와 연산,다섯 자리 이상의 수,A-4-7,4-1
1,2049,큰 수 > 뛰어 세기를 해 볼까요,수와 연산,다섯 자리 이상의 수,A-4-9,4-1
2,2050,큰 수 > 수의 크기를 비교해 볼까요 / 큰 수 > 수의 크기를 비교해 볼까요,수와 연산,다섯 자리 이상의 수,A-4-10,4-1
3,2052,각도 > 어느 각이 더 클까요,도형과 측정(도형),도형의 기초,C-3-3,4-1
4,2053,각도 > 각의 크기는 얼마일까요,도형과 측정(측정),각도( ° ),D-8-1,4-1
...,...,...,...,...,...,...
637,10741,제곱근과 실수 > 제곱근과 실수 > 무리수와 실수,수와 연산,제곱근과 실수,정보 없음,9-1
638,5141,제곱근과 실수 > 제곱근과 실수 > 무리수와 실수,수와 연산,제곱근과 실수,정보 없음,9-1
639,5142,제곱근과 실수 > 제곱근과 실수 > 실수의 대소 관계,수와 연산,제곱근과 실수,정보 없음,9-1
640,5921,제곱근과 실수 > 제곱근과 실수 > 무리수와 실수,수와 연산,제곱근과 실수,정보 없음,9-1


학생이 맞춘 knowledgeTag와 틀린 knowledgeTag의 빈도수가 많은걸 가져와 이후, 이전 학습을 추천해줌

In [11]:
import pandas as pd
from collections import defaultdict
import re
import numpy as np


# NaN 값을 빈 문자열로 대체
label_math_ele['from_chapter_name'].fillna('', inplace=True)
label_math_ele['to_chapter_name'].fillna('', inplace=True)


def extract_semester(semester_str):
    if pd.isna(semester_str):
        return ''
    match = re.search(r'(\d+)-(\d+)', semester_str)
    return (int(match.group(1)), int(match.group(2))) if match else (None, None)


def parse_series_order(order_str):
    if not order_str:  # None 또는 빈 문자열일 때
        return ('ZZZ', float('inf'), float('inf'))  # Default value (정렬 우선순위 가장 낮음)
    match = re.match(r'([A-Z]+)-(\d+)-(\d+)', order_str)
    if match:
        return (match.group(1), int(match.group(2)), int(match.group(3)))
    return ('ZZZ', float('inf'), float('inf'))  # Default value for malformed input (정렬 우선순위 가장 낮음)


def get_unique_id_name_pairs(df):
    to_id_name = dict(zip(df['to_id'], df['to_chapter_name']))
    from_id_name = dict(zip(df['from_id'], df['from_chapter_name']))
    all_ids = set(to_id_name.keys()).union(set(from_id_name.keys()))
    unique_id_name_pairs = {}
    unique_id_semesters = {}
    for id_ in all_ids:
        to_name = to_id_name.get(id_, '')
        from_name = from_id_name.get(id_, '')
        if to_name and from_name:
            unique_id_name_pairs[id_] = f'{to_name} / {from_name}'
        elif to_name:
            unique_id_name_pairs[id_] = to_name
        elif from_name:
            unique_id_name_pairs[id_] = from_name
        from_semester = df[df['from_id'] == id_]['from_semester'].values
        to_semester = df[df['to_id'] == id_]['to_semester'].values
        if len(from_semester) > 0:
            unique_id_semesters[id_] = extract_semester(from_semester[0])
        elif len(to_semester) > 0:
            unique_id_semesters[id_] = extract_semester(to_semester[0])
        else:
            unique_id_semesters[id_] = (None, None)
    return unique_id_name_pairs, unique_id_semesters


# 본개념과 ID-이름 쌍, 학기 정보 가져오기
unique_id_name_pairs, unique_id_semesters = get_unique_id_name_pairs(label_math_ele)


# education_2022 데이터를 ID를 기준으로 매칭할 수 있도록 딕셔너리 생성
education_mapping = education_2022.set_index('ID').to_dict('index')
predecessors = defaultdict(list)
successors = defaultdict(list)
to_chapter_names = dict(zip(label_math_ele['to_id'], label_math_ele['to_chapter_name']))
from_chapter_names = dict(zip(label_math_ele['from_id'], label_math_ele['from_chapter_name']))
for _, row in label_math_ele.iterrows():
    from_id = row['from_id']
    to_id = row['to_id']
    predecessors[from_id].append(to_id)
    successors[to_id].append(from_id)


# 특정 학습자에 대한 맞은 문제와 틀린 문제 집계
def analyze_student_performance(learner_id, df):
    student_df = df[df['learnerID'] == learner_id].copy()
    if student_df.empty:
        print(f"학습자 {learner_id}의 데이터가 없습니다.")
        return None, None
    # 'answerCode'의 데이터 타입 확인 및 변환
    if student_df['answerCode'].dtype == 'object':
        student_df.loc[:, 'answerCode'] = pd.to_numeric(student_df['answerCode'], errors='coerce')
    # 맞은 문제와 틀린 문제 분류
    correct_df = student_df[student_df['answerCode'] == 1]
    incorrect_df = student_df[student_df['answerCode'] == 0]
    # 각 문제의 knowledgeTag 수집
    correct_knowledge_tags = correct_df['knowledgeTag'].dropna()
    incorrect_knowledge_tags = incorrect_df['knowledgeTag'].dropna()
    return correct_knowledge_tags.tolist(), incorrect_knowledge_tags.tolist()


# 빈도수가 가장 높은 태그를 찾는 함수
def get_most_common_tag(tags_list):
    all_tags = [tag for sublist in tags_list for tag in sublist]  # 중첩 리스트를 평탄화
    if all_tags:  # 리스트가 비어있지 않은 경우
        return pd.Series(all_tags).value_counts().idxmax()
    return None


# 특정 개념에 대한 본개념, 선수개념, 후속개념을 출력하는 함수
def get_concepts(node_id):
    main_concept = {
        '본개념': node_id,
        '본개념_Chapter_Name': unique_id_name_pairs.get(node_id, '정보 없음'),
        '학년-학기': unique_id_semesters.get(node_id, (None, None))
    }
    # 선수학습과 후속학습을 '계열화' 기준으로 정렬
    main_concept['선수학습'] = sorted(
        predecessors.get(node_id, []),
        key=lambda x: parse_series_order(education_mapping.get(x, {}).get('계열화', ''))
    , reverse=True)
    main_concept['후속학습'] = sorted(
        successors.get(node_id, []),
        key=lambda x: parse_series_order(education_mapping.get(x, {}).get('계열화', ''))
    )
    return main_concept


def add_education_info(df, id_col):
    df['영역명'] = df[id_col].apply(lambda x: education_mapping.get(x, {}).get('영역명', '정보 없음'))
    df['내용요소'] = df[id_col].apply(lambda x: education_mapping.get(x, {}).get('내용요소', '정보 없음'))
    df['계열화'] = df[id_col].apply(lambda x: education_mapping.get(x, {}).get('계열화', '정보 없음'))
    return df


# NaN 값을 특정 문자열로 대체
def replace_nan_with_string(df, replace_str='정보 없음'):
    return df.replace({np.nan: replace_str})

def get_concepts_df(node_id):
    # Get concept data
    concepts = get_concepts(node_id)
    
    # Create main concept DataFrame
    main_concept_df = pd.DataFrame([{
        '본개념_ID': concepts['본개념'],
        '본개념_Chapter_Name': concepts['본개념_Chapter_Name'],
        '학년-학기': f"{concepts['학년-학기'][0]}-{concepts['학년-학기'][1]}"
    }])
    
    # Add additional education info
    main_concept_df = add_education_info(main_concept_df, '본개념_ID')
    
    # Replace NaN with specific string
    main_concept_df = replace_nan_with_string(main_concept_df)
    print("Main Concept DataFrame:", main_concept_df)  # For debugging
    
    # Create predecessors DataFrame
    predecessors_df = pd.DataFrame({
        '선수학습_ID': concepts['선수학습'],
        '선수학습_Chapter_Name': [unique_id_name_pairs.get(id, '정보 없음') for id in concepts['선수학습']],
        '학년-학기': [f"{unique_id_semesters.get(id, (None, None))[0]}-{unique_id_semesters.get(id, (None, None))[1]}" for id in concepts['선수학습']]
    })
    
    # Add additional education info
    predecessors_df = add_education_info(predecessors_df, '선수학습_ID')
    
    # Replace NaN with specific string
    predecessors_df = replace_nan_with_string(predecessors_df)
    print("Predecessors DataFrame:", predecessors_df)  # For debugging
    
    # Create successors DataFrame
    successors_df = pd.DataFrame({
        '후속학습_ID': concepts['후속학습'],
        '후속학습_Chapter_Name': [unique_id_name_pairs.get(id, '정보 없음') for id in concepts['후속학습']],
        '학년-학기': [f"{unique_id_semesters.get(id, (None, None))[0]}-{unique_id_semesters.get(id, (None, None))[1]}" for id in concepts['후속학습']]
    })
    
    # Add additional education info
    successors_df = add_education_info(successors_df, '후속학습_ID')
    
    # Replace NaN with specific string
    successors_df = replace_nan_with_string(successors_df)
    print("Successors DataFrame:", successors_df)  # For debugging
    
    return main_concept_df, predecessors_df, successors_df


# 사용자로부터 학습자 ID 입력 받기
learner_id_input = input("학습자 ID를 입력하세요: ").strip()
# 학습자 ID에 대한 데이터 분석
correct_knowledge_tags, incorrect_knowledge_tags = analyze_student_performance(learner_id_input, merged_df)
if correct_knowledge_tags is not None and incorrect_knowledge_tags is not None:
    # 결과에서 가장 빈도수 높은 태그 추출
    most_common_correct_tag = get_most_common_tag([correct_knowledge_tags])
    most_common_incorrect_tag = get_most_common_tag([incorrect_knowledge_tags])
    print(f"\n학습자 {learner_id_input}의 맞은 문제에서 가장 빈도수가 높은 knowledgeTag: {most_common_correct_tag}")
    print("맞은 문제 관련 본개념, 후속개념 정보:")
    correct_main_concept_df, _, correct_successors_df = get_concepts_df(most_common_correct_tag)  # 선수학습을 무시하기 위해 '_' 사용
    # 틀린 문제의 knowledgeTag로 개념 정보 출력
    print(f"\n학습자 {learner_id_input}의 틀린 문제에서 가장 빈도수가 높은 knowledgeTag: {most_common_incorrect_tag}")
    print("틀린 문제 관련 본개념, 선수개념 정보:")
    incorrect_main_concept_df, incorrect_predecessors_df, _ = get_concepts_df(most_common_incorrect_tag)  # 후속학습을 무시하기 위해 '_' 사용

C:\Users\user\AppData\Local\Temp\ipykernel_22844\641016027.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_math_ele['from_chapter_name'].fillna('', inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_22844\641016027.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_math_ele['to_chapter_name'].fillna('', inplace=True)



학습자 A030000005의 맞은 문제에서 가장 빈도수가 높은 knowledgeTag: 439
맞은 문제 관련 본개념, 후속개념 정보:
Main Concept DataFrame:    본개념_ID                                   본개념_Chapter_Name 학년-학기  \
0     439  길이와 시간 > 시간은 어떻게 더하고 뺄까요 (1) / 길이와 시간 > 시간은 어떻...   3-1   

          영역명       내용요소    계열화  
0  도형과 측정(측정)  시각과 시간(초)  D-4-2  
Predecessors DataFrame:    선수학습_ID                                  선수학습_Chapter_Name 학년-학기  \
0      438  길이와 시간 > 1분보다 작은 단위는 무엇일까요 / 길이와 시간 > 1분보다 작은 ...   3-1   
1     8132          시각과 시간 > 1시간을 알아볼까요 / 시각과 시간 > 1시간을 알아볼까요   2-2   
2     8130  시각과 시간 > 몇 시 몇 분을 알아볼까요 (2) / 시각과 시간 > 몇 시 몇 분...   2-2   
3     8129  시각과 시간 > 몇 시 몇 분을 알아볼까요 (1) / 시각과 시간 > 몇 시 몇 분...   2-2   

          영역명           내용요소    계열화  
0  도형과 측정(측정)      시각과 시간(초)  D-4-1  
1  도형과 측정(측정)  시각과 시간 (시, 분)  D-2-6  
2  도형과 측정(측정)  시각과 시간 (시, 분)  D-2-4  
3  도형과 측정(측정)  시각과 시간 (시, 분)  D-2-3  
Successors DataFrame:    후속학습_ID             후속학습_Chapter_Name 학년-학기         영역명       내용요소    계열화
0      443  길이와 시간 > 